In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
%pwd

'e:\\Projects\\Text Summarization\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\Projects\\Text Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from src.Text_Summarization.constants import *
from src.Text_Summarization.utils.common import read_yaml, create_directories

In [6]:
class ConfigrationManager:
    def __init__(self, 
                 params_filepath= PARAMS_FILE_PATH,
                 config_filepath= CONFIG_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        #print(config.root_dir)
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [7]:
import os
import sys
from transformers import AutoTokenizer
from src.Text_Summarization.logging import logger
from src.Text_Summarization.exception import CustomException
from datasets import load_dataset, load_from_disk

e:\Projects\Text Summarization\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Projects\Text Summarization\venv\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Projects\Text Summarization\venv\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Projects\T

[ 2024-08-13 15:27:28,248 ] 58 datasets - INFO - PyTorch version 2.4.0+cu118 available.


In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsun = load_from_disk(os.path.join(BASE_DIR,self.config.data_path))
        dataset_samsun_pt = dataset_samsun.map(self.convert_example_to_features, batched = True)
        dataset_samsun_pt.save_to_disk(os.path.join(BASE_DIR,self.config.root_dir,"samsun_dataset"))

In [11]:
try:
    config = ConfigrationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-08-13 15:30:12,555 ] 23 TextSummarizationLogger - INFO - yaml file: E:\Projects\Text Summarization\config\config.yaml loaded successfully
[ 2024-08-13 15:30:12,563 ] 23 TextSummarizationLogger - INFO - yaml file: params.yaml loaded successfully
[ 2024-08-13 15:30:12,565 ] 41 TextSummarizationLogger - INFO - created directory at: artifacts
[ 2024-08-13 15:30:12,566 ] 41 TextSummarizationLogger - INFO - created directory at: artifacts/data_transformation


CustomException: Error occured in Python Script name [C:\Users\HP\AppData\Local\Temp\ipykernel_35188\1799740383.py] line number [5] error message [Directory artifacts/data_ingestion/samsun_dataset not found]